In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import statsmodels.api as sm
import STAT5102_2023 as stat
from stepwise_regression import step_reg

In [2]:
# input the stock ticker we wana study
stock_ticker = "TSLA"
current_date = '2025-03-28'
maturity_date = '2025-04-17'
direction = 'call' 

In [3]:
# read option chain file
df_raw = pd.read_excel('Combined_' + stock_ticker + '_options_15-Apr-25.xlsx')
df = df_raw.copy()

In [ ]:
# Function to get option chain in the dataframe
# Select option chain with specific current date and maturity date
def get_option_chain(df_input, symbol_input, direction, current_date_input, maturity_date_input=None):
    # Ignore maturity date when the user doesn't put in a maturity date
    condition = (df_input["Symbol"] == symbol_input) & (df_input['Direction'] == direction) & (df_input['Current Date'] == current_date_input)
    # Filter with maturity date when the user put in a maturity date
    if not maturity_date_input is None:
        condition = condition & (df_input['Expiration Date'] == maturity_date_input)
    df_res = df_input[condition]
    return df_res

# Function to perform linear regression with list of columns as input, using sm.OLS
def linear_regression(df_input, y_col, x_cols):
    df = df_input
    X = df[x_cols]
    y = df[y_col]
    X = sm.add_constant(X)  # Adds a constant term to the predictor
    model = sm.OLS(y, X).fit()
    return model

# Define a function to plot the IVM across moneyness with maturity in the legend
def plot_ivm_across_moneyness(df_input, symbol_input, direction_input, current_date_input):

    # Filter the data for the specified stock, direction, and date
    df_target = get_option_chain(df_input, symbol_input, direction_input, current_date_input)
    
    # Group by maturity (maturity_years)
    grouped = df_target.groupby('maturity_years')
    
    # Create the plot
    plt.figure(figsize=(10, 6))
    for maturity, group in grouped:
        plt.scatter(group['moneyness'], group['IVM'], label=f'Maturity: {maturity:.2f} years')
    
    # Add labels, title, legend, and grid
    plt.xlabel('Moneyness')
    plt.ylabel('IVM')
    plt.title(f'IVM across Moneyness for {symbol_input} {direction_input} options on {current_date_input}')
    plt.legend(title='Maturity (years)', loc='best')
    plt.grid()
    plt.show()

# Function to plot 3D graph of IVM across moneyness and maturity_years
def plot_ivm_across_moneyness_and_days(df, symbol_input, direction, current_date_input):

    # Get the filtered option chain
    df_target = get_option_chain(df, symbol_input, direction, current_date_input)
    
    # Create the 3D plot
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(df_target['moneyness'], 
               df_target['maturity_years'], 
               df_target['IVM'], 
               color='blue')
    
    # Set labels and title
    ax.set_xlabel('Moneyness')
    ax.set_ylabel('Time to Maturity (in years)')
    ax.set_zlabel('IVM')
    ax.set_title(f'IVM across Moneyness and Time to Maturity for {symbol_input} {direction} options on {current_date_input}')
    
    # Show the plot
    plt.show()

# Function to input current stock price, strike, maturity date and direction, then output moneyess, maturity_years,'moneyness_2', 'maturity_2', 'moneyness_maturity' in list
def get_warrant_moneyness_expiration(current_stock_price, strike, maturity_date_input, current_date_input):
    # Convert dates to datetime format
    current_date = pd.to_datetime(current_date_input)
    maturity_date = pd.to_datetime(maturity_date_input)
    
    # Calculate moneyness and maturity_years
    moneyness = current_stock_price / strike
    time_to_expiration_days = (maturity_date - current_date).days
    time_to_expiration_years = time_to_expiration_days / 365
    
    # Calculate squares and products
    moneyness_2 = moneyness ** 2
    maturity_2 = time_to_expiration_years ** 2
    moneyness_maturity = moneyness * time_to_expiration_years
    
    return [moneyness, time_to_expiration_years, moneyness_2, maturity_2, moneyness_maturity]


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11199 entries, 0 to 11198
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Ticker                    11199 non-null  object 
 1   Symbol                    11199 non-null  object 
 2   Expiration Date           11199 non-null  object 
 3   Strike                    11199 non-null  float64
 4   Bid                       11199 non-null  float64
 5   Ask                       11199 non-null  float64
 6   Last                      11199 non-null  float64
 7   IVM                       11199 non-null  float64
 8   Direction                 11199 non-null  object 
 9   Current Date              11199 non-null  object 
 10  Current Date Stock Price  11199 non-null  float64
dtypes: float64(6), object(5)
memory usage: 962.5+ KB


In [6]:
#show unique values in "Expiration Date" in ascending order
date_count = df["Expiration Date"].value_counts().reset_index()
date_count["Expiration Date"] = pd.to_datetime(date_count["Expiration Date"])
date_count.set_index("Expiration Date", inplace=True)
date_count = date_count.sort_index()
date_count

,count
Expiration Date,
2025-01-17,880
2025-02-21,2720
2025-03-21,3359
2025-04-17,2880
2025-04-25,160
2025-05-16,1200


In [7]:
# Remove rows where "IVM" is 0
df = df[df['IVM'] != 0]
df.head()

,Ticker,Symbol,Expiration Date,Strike,Bid,Ask,Last,IVM,Direction,Current Date,Current Date Stock Price
1,TSLA 4/17/25 P207.5,TSLA,17-Apr-2025,207.5,0.120000,0.140000,0.12,105.147621,put,15-Apr-2025,254.110001
2,TSLA 4/17/25 P262.5,TSLA,17-Apr-2025,262.5,11.400000,11.600000,10.70,62.645725,put,15-Apr-2025,254.110001
3,TSLA 4/17/25 P265,TSLA,17-Apr-2025,265.0,13.350000,13.600000,12.48,63.222397,put,15-Apr-2025,254.110001
4,TSLA 4/17/25 P267.5,TSLA,17-Apr-2025,267.5,15.450000,15.700000,14.60,64.077888,put,15-Apr-2025,254.110001
5,TSLA 4/17/25 P270,TSLA,17-Apr-2025,270.0,17.550003,17.899994,16.75,64.198608,put,15-Apr-2025,254.110001


In [8]:
# Add columns for regression

# Ensure 'Expiration Date' and 'Current Date' are in datetime format
df['Expiration Date'] = pd.to_datetime(df['Expiration Date'])
df['Current Date'] = pd.to_datetime(df['Current Date'])

# Add moneyness, time to maturity in days and time to maturity in years
df['moneyness'] = df['Current Date Stock Price'] / df['Strike']
df['maturity_days'] = (df['Expiration Date'] - df['Current Date']).dt.days
df['maturity_years'] = df['maturity_days'] / 365

# Transform moneyness, time_to_maturity_days, maturity_years and IVM to between 0 and 1
cols_to_standardize = ['moneyness', 'maturity_days', "maturity_years", "IVM"]
for col in cols_to_standardize:
       df[col] =  (df[col] - np.min(df[col].values)) / (np.max(df[col].values) - np.min(df[col].values))


col_order = ['Ticker', 'Symbol', 'Direction', 'Current Date', 'Expiration Date', 'maturity_days', 'maturity_years', 'Current Date Stock Price', 'Strike', 'Bid', 'Ask', 'Last',
       'IVM', 'moneyness']
df = df.loc[:, col_order]

df.describe()

,Current Date,Expiration Date,maturity_days,maturity_years,Current Date Stock Price,Strike,Bid,Ask,Last,IVM,moneyness
count,10704,10704,10704.000000,10704.000000,10704.000000,10704.000000,10704.000000,10704.000000,10704.000000,10704.000000,10704.000000
mean,2025-02-22 21:17:21.255605248,2025-03-23 01:33:37.937219840,0.402542,0.402542,322.979204,323.440069,30.143513,30.863887,29.052094,0.016370,0.328525
min,2025-01-02 00:00:00,2025-01-17 00:00:00,0.000000,0.000000,221.860001,125.000000,0.000000,0.010000,0.000000,0.000000,0.000000
25%,2025-01-28 00:00:00,2025-02-21 00:00:00,0.185714,0.185714,263.450012,265.000000,5.950000,6.100000,5.340000,0.008930,0.231087
50%,2025-02-24 00:00:00,2025-03-21 00:00:00,0.414286,0.414286,328.500000,320.000000,19.850006,20.149994,18.714998,0.010699,0.311173
75%,2025-03-21 00:00:00,2025-04-17 00:00:00,0.614286,0.614286,394.739990,375.000000,45.199997,46.362503,43.709999,0.013085,0.406874
max,2025-04-15 00:00:00,2025-05-16 00:00:00,1.000000,1.000000,428.220001,525.000000,192.949997,194.199997,188.250000,1.000000,1.000000
std,NaN,NaN,0.243292,0.243292,67.356963,73.908591,30.793049,31.526860,30.267034,0.046569,0.143760


In [ ]:
# Add columns for moneyness squared, moneyness cubed and moneyness quadrupled
df['moneyness_2'], df["moneyness_3"] = df['moneyness'] ** 2, df["moneyness"] ** 3

# Add columns for maturity squared, maturity cubed and maturity quadrupled
df['maturity_2'], df['maturity_3'] = df['maturity_years'] ** 2, df['maturity_years'] ** 3

# Add interaction term of moneyness and maturity
df['moneyness_maturity'] = df['moneyness'] * df['maturity_years']

# Add inverse of moneyness
df['moneyness_inv1'] = 1 / df['moneyness']

# Add inverse of maturity
df['maturity_inv1'] = 1 / df['maturity_years']


In [ ]:
# Select the option chain for a specific stock, direction, and current date and maturity date
df_target = get_option_chain(df, stock_ticker, direction, current_date, maturity_date)

# Define the dependent and independent variables
Y = "IVM"
X = [col for col in df_target.columns if not col in ("Ticker", "Symbol", "Current Date", "Expiration Date", "maturity_days", "Current Date Stock Price", "Strike", "Bid", "Ask", "Last", "Direction", "IVM")]
print(stat.stepwise_regresson(X, Y, df_target))

{'report': [('maturity_3', 1.1296028696098966e-50, 1), ('maturity_inv1', 1.1296028696099127e-50, 1), ('maturity_years', 1.129602869609929e-50, 1), ('maturity_2', 1.129602869609929e-50, 1), ('moneyness_3', 0.015210060405264302, 1), ('moneyness_inv1', 1.384401263968326e-32, 1)], 'selected': ['maturity_3', 'maturity_inv1', 'maturity_years', 'maturity_2', 'moneyness_3', 'moneyness_inv1']}


In [ ]:
# Select the option chain for a specific stock, direction, and current date and maturity date
df_target = get_option_chain(df, stock_ticker, direction, current_date, maturity_date)

# Define the dependent and independent variables
dependent_var = 'log_IVM'  # USE log IVM
independent_vars = ['moneyness', 'maturity_years', 'moneyness_2', 'maturity_2', 'moneyness_maturity'] # we set the regressor

# Perform linear regression
regression_summary = linear_regression(df_target, dependent_var, independent_vars).summary()

# Display the regression summary
print(regression_summary)

In [ ]:
# print(df_new[['IVM']])

In [ ]:
plot_ivm_across_moneyness_and_days(df, stock_ticker, 'call', current_date)  #show call volatility smile

In [ ]:
plot_ivm_across_moneyness_and_days(df, stock_ticker, 'put', current_date)  # show put volatility smile

In [ ]:
plot_ivm_across_moneyness(df, stock_ticker, 'call', '2025-03-28')  # show call volatility smileb

In [ ]:
plot_ivm_across_moneyness(df, stock_ticker, 'put', '2025-03-28')  # show put volatility smile

We test with the following regression models, letting $K$ be moneyness, $T$ be time to maturity, $K*T$ be moneyness_maturity, $K^2$ be moneyness squared and $T^2$ be maturity squared.

- Model 1: $K, T, K*T$
- Model 2: $K, T, K^2$
- Model 3: $K, T, T^2$
- Model 4: $K, T, K^2, K*T$
- Model 5: $K, T, T^2, K*T$
- Model 6: $K, T, K^2, T^2, K*T$

In [ ]:
# Function to perform linear regression with list of columns as input, using sm.OLS
# model1: K,T,K*T

independent_vars_one = ['moneyness', 'maturity_years', 'moneyness_maturity'] # model1: K,T,K*T
model = linear_regression(df_target, dependent_var, independent_vars_one)


print(model.summary())

In [ ]:
# Function to perform linear regression with list of columns as input, using sm.OLS
# model2: K,T,K^2

independent_vars_two = ['moneyness', 'maturity_years', 'moneyness_2'] # model2: K,T,K^2
model = linear_regression(df_target, dependent_var, independent_vars_two)


print(model.summary())

In [ ]:
# Function to perform linear regression with list of columns as input, using sm.OLS
# model3: K,T,T^2

independent_vars_three = ['moneyness', 'maturity_years', 'maturity_2'] # model3: K,T,T^2
model = linear_regression(df_target, dependent_var, independent_vars_three)


print(model.summary())

In [ ]:
# Function to perform linear regression with list of columns as input, using sm.OLS
# model4: K,T,K^2,K*T

independent_vars_four = ['moneyness', 'maturity_years', 'moneyness_2', 'moneyness_maturity'] # model4: K,T,K^2,K*T
model = linear_regression(df_target, dependent_var, independent_vars_four)


print(model.summary())

In [ ]:
# Function to perform linear regression with list of columns as input, using sm.OLS
# model5 K,T,T^2,K*T

independent_vars_five = ['moneyness', 'maturity_years','maturity_2','moneyness_maturity'] # model5 K,T,T^2,K*T
model = linear_regression(df_target, dependent_var, independent_vars_five)


print(model.summary())

In [ ]:
# Function to perform linear regression with list of columns as input, using sm.OLS
# model6:K,T,K^2,T^2,K*T

independent_vars_six = ['moneyness', 'maturity_years', 'moneyness_2', 'maturity_2','moneyness_maturity'] # model6:K,T,K^2,T^2,K*T
model = linear_regression(df_target, dependent_var, independent_vars_six)


print(model.summary())

In [ ]:
# Function to perform linear regression with list of columns as input, using sm.OLS
#prototype regression model

independent_vars = ['moneyness', 'maturity_years', 'moneyness_2', 'maturity_2', 'moneyness_maturity']
model = linear_regression(df_target, dependent_var, independent_vars)


print(model.summary())

In [ ]:
# output warrant moneyness, maturity_years, 'moneyness_2', 'maturity_2', 'moneyness_maturity' in list
#******INPUT warrant data HERE ******
current_stock_price = 277.13
strike = 350
maturity_date = '2025-05-21'
current_date = '2025-03-27'
warrant_moneyness_expiration = get_warrant_moneyness_expiration(current_stock_price, strike, maturity_date, current_date)
print(warrant_moneyness_expiration)

In [ ]:
# apply the regression model to calcualte warrant IVM
X_new = pd.DataFrame([warrant_moneyness_expiration], columns=independent_vars)
predictions = model.predict(X_new)
print(predictions)


In [ ]:
# prediction of in sample data, IVM from bloomberg and IVM predicted from the regression 

# df_target['Predicted_IVM'] = math.exp(model.predict(df_target[independent_vars]))

df_target['Predicted_IVM'] = np.exp(model.predict(df_target[independent_vars]))
print(df_target[['IVM', 'Predicted_IVM']])

In [ ]:
# Extract only the date part from 'Current Date' and 'Expiration Date'
current_dates = [date.strftime('%Y-%m-%d') for date in df['Current Date'].dt.date.unique()]
maturity_dates = [date.strftime('%Y-%m-%d') for date in df['Expiration Date'].dt.date.unique()]

# Print the results
print(current_dates)
print(maturity_dates)



In [ ]:
independent_vars_one = ['moneyness', 'maturity_years', 'moneyness_maturity'] # model1: K,T,K*T
independent_vars_two = ['moneyness', 'maturity_years', 'moneyness_2'] # model2: K,T,K^2
independent_vars_three = ['moneyness', 'maturity_years', 'maturity_2'] # model3: K,T,T^2
independent_vars_four = ['moneyness', 'maturity_years','moneyness_2','moneyness_maturity'] # model4: K,T,K^2,K*T
independent_vars_five = ['moneyness', 'maturity_years','maturity_2','moneyness_maturity'] # model5 K,T,T^2,K*T
independent_vars_six = ['moneyness', 'maturity_years','moneyness_2','maturity_2','moneyness_maturity'] # model6:K,T,K^2,T^2,K*T

ress = [] # store the regression result in a list, current_date, maturity_date, model1, model2, model3, model4, model5, model6

for current_date in current_dates:
    res = []
    for maturity_date in maturity_dates:
        df_target = get_option_chain(df, stock_ticker, 'call', current_date, maturity_date)
        if df_target.empty:
            continue
        dependent_var = 'IVM'
        model1 = linear_regression(df_target, dependent_var, independent_vars_one)
        model2 = linear_regression(df_target, dependent_var, independent_vars_two)
        model3 = linear_regression(df_target, dependent_var, independent_vars_three)
        model4 = linear_regression(df_target, dependent_var, independent_vars_four)
        model5 = linear_regression(df_target, dependent_var, independent_vars_five)
        model6 = linear_regression(df_target, dependent_var, independent_vars_six)

        res.append([current_date, maturity_date,
                    model1.rsquared_adj,
                    model2.rsquared_adj,
                    model3.rsquared_adj,
                    model4.rsquared_adj,
                    model5.rsquared_adj,
                    model6.rsquared_adj])
    ress.append(res)

In [ ]:
# Flatten the nested list `ress`
flattened_ress = [item for sublist in ress for item in sublist]

# Create a DataFrame
columns = ['Current Date', 'Maturity Date', 'Model1 adj_R-squared', 'Model2 adj_R-squared', 
           'Model3 adj_R-squared', 'Model4 adj_R-squared', 'Model5 adj_R-squared', 'Model6 adj_R-squared']
df_regression_results_adj_rsquare = pd.DataFrame(flattened_ress, columns=columns)

# Display the DataFrame
print(df_regression_results_adj_rsquare)
df_regression_results_adj_rsquare.to_excel(stock_ticker+'_log_regression_results_adj_rsquare.xlsx', index=False)